In [3]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os

# Load the combined model
cnn = tf.keras.models.load_model("D:/Users/rafae/Desktop/trained_combined_signature_model_with_sobel.h5")

# Check if the correct number of command-line arguments is provided
if len(sys.argv) != 2:
    print("Usage: python script.py <image_path>")
    sys.exit(1)

# Get the image path from the command-line arguments
image_path = sys.argv[1]

# Read and display the image
img = cv2.imread(image_path)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title("Signature Image")
plt.xticks([])
plt.yticks([])
plt.show()

# Load and preprocess the image for prediction
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.expand_dims(input_arr, axis=0)  # Expand dimensions to create a batch of size 1

    # Convert to grayscale
    gray = tf.image.rgb_to_grayscale(input_arr)
    gray = tf.cast(gray, tf.float32)

    # Apply Sobel edge detection
    edges_x = tf.image.sobel_edges(gray)[..., 0]  # Sobel X
    edges_y = tf.image.sobel_edges(gray)[..., 1]  # Sobel Y
    edges = tf.sqrt(tf.square(edges_x) + tf.square(edges_y))

    # Normalize the edges
    edges = (edges - tf.reduce_min(edges)) / (tf.reduce_max(edges) - tf.reduce_min(edges) + 1e-6)

    # Resize to match the input size of the CNN
    edges = tf.image.resize_with_crop_or_pad(edges, 64, 64)

    # Expand dimensions to match expected shape for CNN
    edges = tf.expand_dims(edges, axis=-1)

    return edges

edges = preprocess_image(image_path)

# Make prediction
predictions = cnn.predict(edges)

# Get the directory where this script is located
script_dir = os.path.dirname(os.path.abspath(__file__))

# Construct dataset paths relative to the script's directory
directories = {
    "fraud_testing": os.path.join(script_dir, 'signature/fraud/testing'),
    "legit_testing": os.path.join(script_dir, 'signature/legit/testing')
}

# Scan fraud testing directory to get fraud class names
fraud_class_names = sorted([name for name in os.listdir(directories["fraud_testing"]) if os.path.isdir(os.path.join(directories["fraud_testing"], name))])

# Scan legit testing directory to get legit class names
legit_class_names = sorted([name for name in os.listdir(directories["legit_testing"]) if os.path.isdir(os.path.join(directories["legit_testing"], name))])

# Combine both fraud and legit class names with prefix
class_names = ['fraud_' + name for name in fraud_class_names] + ['legit_' + name for name in legit_class_names]

# Debug: Print the combined class names and predicted index
print("Combined Class Names:", class_names)
predicted_index = np.argmax(predictions[0])
print("Predicted Index:", predicted_index)

# Determine the predicted class and label
predicted_class_name = class_names[predicted_index]
if 'fraud_' in predicted_class_name:
    predicted_class = "fraud"
    predicted_label = predicted_class_name.replace('fraud_', '')
else:
    predicted_class = "legit"
    predicted_label = predicted_class_name.replace('legit_', '')

# Display the image again with prediction result
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title(f"Predicted signature: {predicted_class}, Predicted person: {predicted_label}")
plt.xticks([])
plt.yticks([])
plt.show()

# Print the predicted class and label
print("Predicted signature:", predicted_class)
print("Predicted person:", predicted_label)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
